# `Maze` Class

Together, we read through the `Maze` class. This is the code responsible for reading a maze file, validating and parsing its contents, and producing visualizations. You can find the whole class [here](../src/Maze.py).

## The Less Important Bits

Some of the code in `Maze` is "utility" code that we didn't spend much time considering:
  - How to open and read the contents of text file into memory
  - How it "prints" the maze it has parsed 
  - How (using the `Pillow` library) it can "draw" the maze

## `__init__`

We did spend a little time working through the `__init__` method. It's an example of a more involved initialization process. In particular, we discussed:
  - how it calls another class method, `_validate`, to ensure that the maze it has read in is valid
  - how it calls another class method, `_parse`, to build up its "internal" 2D array (list of lists) representation of the maze

Here's the `__init__` method in full:
```python
def __init__(self, maze_name):
        self.path_to_maze_file = f"../mazes/{maze_name}.txt"

        # Read maze file
        with open(self.path_to_maze_file) as f:
            self.maze_raw = self._validate(f.read())

        self.maze_linesplit = self.maze_raw.splitlines()
        self.height = len(self.maze_linesplit)
        self.width = max(len(line) for line in self.maze_linesplit)
        self.start, self.goal, self.walls = self._parse(self.maze_linesplit)
```

## "Parsing" the Maze, Building the Internal Representation

Critically, the `Maze` class is responsible for turning a bunch of spaces, #'s, an A and a B arranged in lines of text into some data structure we can use *algorithmically* to solve the maze.

Here's the code that does it:
```python
def _parse(self, linesplit):
        walls = []
        start = None
        goal = None

        for i in range(self.height):
            row = []
            for j in range(self.width):
                try:
                    if linesplit[i][j] == "A":
                        start = (i, j)
                        row.append(False)
                    elif linesplit[i][j] == "B":
                        goal = (i, j)
                        row.append(False)
                    elif linesplit[i][j] == " ":
                        row.append(False)
                    else:
                        row.append(True)
                except IndexError:
                    row.append(False)
            walls.append(row)

        return (start, goal, walls)
```

We use nested `for` loops -- a `for` loop *inside* another `for` loop -- to build up the representation.
  - The "outer" loop takes a line of text at a time. Each line represents a "row" of the maze.
  - For each of those rows, the "inner" loop looks at each character. The `if-elif-elif-else` control structure determines if the current character is the starting point, the goal, a pathway, or a wall and adds a `True` (if it's a wall) or a `False` (if it's not) to the row array.
  - Each parsed row becomes one of the "inner" lists. The list of those (row) lists is the 2D array that represents the maze.

## `get_neighbors` Method

You might have noticed that the `Maze` class is not responsible for solving itself. It's only a representation of the maze, a data structure suited to storing all the relevant details of a maze alongside some functions that operate on that representation. (If you want to represent the solution or explored paths when the maze is printed or drawn, they have to be supplied from outside the class.)

But there is one method that we'll need when solving the maze: `get_neighbors`. The `get_neighbors` method takes in a cell's coordinates and returns a list of all the possible next steps. Here's the code:
```python
def get_neighbors(self, coordinates):
        row, col = coordinates
        possible_neighbors = [
            ('up', (row - 1, col)),
            ('down', (row + 1, col)),
            ('left', (row, col - 1)),
            ('right', (row, col + 1)),
        ]
     
        neighbors = []
        for direction, (r, c) in possible_neighbors:
            if 0 <= r < self.height and 0 <= c < self.width and not self.walls[r][c]:
                neighbors.append((direction, (r, c)))

        return neighbors
```

It first gets the coordinates one cell above, below, to the left, and to the right of the supplied coordinates. Our 2D array representation makes that easy: look one row above or below, one index left or right.

It then loops through those four coordiantes and filters out any that:
  - are outside the boundaries of the maze
  - are walls

What's left is returned. These are all the possible next moves the algorithm will need to consider.